In [2]:
import os
import json
import random
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

In [3]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [6]:
class myDataset(Dataset):
    def __init__(self, df):
        self.seq_list1 = list(df['Wspd_seq'])
        self.seq_list2 = list(df['Patv_seq'])
        
        self.seq_list3 = list(df['Etmp_seq'])
        self.seq_list4 = list(df['Itmp_seq'])
        
        self.seq_list5 = list(df['Patv_space'])

        
        self.label_list = df.target.values

    def __getitem__(self, index):
        
        seq = np.vstack((self.seq_list1[index], self.seq_list2[index], self.seq_list3[index], self.seq_list4[index]))
        seq = np.array(seq).astype('float')
        
        image = np.array(self.seq_list5[index]).astype('float') 
        image.resize(11, 11, 1)

        label = np.array( self.label_list[index] ).astype( 'float' )
        
        seq = torch.tensor(seq, device = 'cuda')
        space_data = torch.tensor(image, device = 'cuda')

        return seq, space_data, label


    def __len__(self):
        return len(self.seq_list1)

In [37]:
# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)
# pool of non-square window
m = nn.MaxPool2d((3, 3))
input = torch.randn(20, 16, 10,10)
output = m(input)

In [38]:
output.shape

torch.Size([20, 16, 3, 3])

In [7]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        
        self.gru = nn.GRU(input_size=4, hidden_size=48, num_layers=2)
        self.dropout = nn.Dropout(0.05)

#         self.Linear = nn.Linear(48+2, 1, bias_attr=True)
        self.Linear = nn.Linear(48+2, 1)

        self.cnnLayer = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1), 
        nn.BatchNorm2d(64),
        nn.GELU(),
        nn.MaxPool2d((3,3)))

        
    def forward(self, X, space_data):

        z = torch.zeros([X.shape[0], 144, X.shape[1]])
        z = z.to(device)
#         print(X.shape)
#         print(z)
#         print(type(x))
#         print(type(X))
#         print(type(z))
        x = torch.concat((torch.transpose(X,2,1),z), axis=1)
#         print(1)
        out1, _ = self.gru(x)
        ou1 = self.dropout(out1)

        cnn_out = self.cnnLayer(space_data)
        print('cnn_out')
        print(cnn_out.shape)
        cnn_out = torch.reshape(cnn_out, (cnn_out.shape[0], 288, -1))
        print(cnn_out.shape)
#         print(cnn_out.shape)
        out2 = self.Linear(torch.concat((out1, cnn_out), 2))

       
        return out2

In [8]:
df_train = pd.read_csv('data/train_data42.csv')

In [9]:
import ast
import math

In [10]:
import ast

def func(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(i)
    return List

In [11]:
df_train['Wspd_seq'] = df_train['Wspd_seq'].apply(func)

In [12]:
df_train['Wspd_seq']

0        [0.29, 0.59, 0.97, 1.38, 1.94, 2.23, 2.54, 1.8...
1        [1.62, 1.15, 1.84, 2.42, 1.72, 1.75, 1.75, 2.9...
2        [2.67, 2.75, 2.65, 2.66, 2.42, 2.31, 2.04, 2.1...
3        [11.25, 10.66, 10.63, 9.57, 10.14, 9.93, 9.13,...
4        [4.52, 3.73, 3.6, 3.49, 3.03, 2.82, 3.56, 3.54...
                               ...                        
37044    [7.07, 7.26, 7.55, 7.57, 7.72, 7.71, 7.05, 7.1...
37045    [3.25, 2.18, 2.09, 1.97, 2.17, 2.37, 2.16, 1.4...
37046    [1.84, 3.88, 4.39, 3.72, 3.91, 3.86, 4.47, 2.9...
37047    [6.32, 5.96, 6.11, 5.59, 5.0, 5.12, 5.3, 5.9, ...
37048    [6.67, 5.74, 6.91, 5.13, 4.51, 6.54, 6.37, 5.6...
Name: Wspd_seq, Length: 37049, dtype: object

In [13]:
df_train['Wspd_seq'] = df_train.apply(
  lambda x: [round(r**3,4) for r in x['Wspd_seq']],
  axis=1,
)

In [14]:
df_train['Wspd_seq']

0        [0.0244, 0.2054, 0.9127, 2.6281, 7.3014, 11.08...
1        [4.2515, 1.5209, 6.2295, 14.1725, 5.0884, 5.35...
2        [19.0342, 20.7969, 18.6096, 18.8211, 14.1725, ...
3        [1423.8281, 1211.3555, 1201.157, 876.4675, 104...
4        [92.3454, 51.8951, 46.656, 42.5085, 27.8181, 2...
                               ...                        
37044    [353.3932, 382.6572, 430.3689, 433.7981, 460.0...
37045    [34.3281, 10.3602, 9.1293, 7.6454, 10.2183, 13...
37046    [6.2295, 58.4111, 84.6045, 51.4788, 59.7765, 5...
37047    [252.436, 211.7087, 228.0991, 174.6769, 125.0,...
37048    [296.741, 189.1192, 329.9394, 135.0057, 91.733...
Name: Wspd_seq, Length: 37049, dtype: object

In [15]:
'----------------------------------------------------------------------------------'

'----------------------------------------------------------------------------------'

In [16]:
df_train

,Wspd_seq,Patv_seq,Etmp_seq,Itmp_seq,Pab1_seq,target,index,TurbID,Patv_space
0,"[0.0244, 0.2054, 0.9127, 2.6281, 7.3014, 11.08...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[21.84, 21.77, 21.81, 21.9, 21.9, 21.87, 21.81...","[26.78, 26.7, 26.55, 26.47, 26.15, 25.75, 25.6...","[90.39, 90.39, 90.39, 90.39, 90.39, 90.39, 90....","[1422.04, 1519.51, 1501.53, 1519.98, 1519.36, ...",21207,14,"[1194.55, 1411.22, 1519.46, 1515.86, 1447.61, ..."
1,"[4.2515, 1.5209, 6.2295, 14.1725, 5.0884, 5.35...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 20.31, 86....","[43.73, 42.19, 42.09, 42.09, 42.02, 41.95, 41....","[42.02, 43.59, 44.67, 44.6, 44.43, 44.28, 44.2...","[43.31, 90.39, 90.39, 90.39, 90.39, 90.39, 90....","[209.79, 247.88, 371.06, 183.73, 194.71, 132.3...",9625,118,"[155.11, 190.08, 153.1, 197.71, 143.08, 155.7,..."
2,"[19.0342, 20.7969, 18.6096, 18.8211, 14.1725, ...","[110.84, 111.16, 103.88, 96.08, 87.81, 88.23, ...","[17.06, 17.05, 16.99, 16.96, 16.79, 16.58, 16....","[17.0, 16.98, 16.84, 16.79, 16.7, 16.63, 16.57...","[-1.99, -1.99, -1.99, -1.99, -1.99, -1.99, -1....","[80.74, 77.03, 100.68, 1.18, 0.0, 0.0, 0.0, 0....",20781,126,"[100.73, 86.67, 83.81, 92.14, 107.54, 117.99, ..."
3,"[1423.8281, 1211.3555, 1201.157, 876.4675, 104...","[1033.06, 1039.58, 1302.54, 1262.94, 1350.97, ...","[37.17, 37.38, 37.4, 37.36, 37.46, 37.7, 37.58...","[41.37, 41.55, 41.87, 42.06, 42.3, 42.5, 42.75...","[9.35, 8.41, 4.81, 1.91, 2.17, 2.04, 1.2, 0.61...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16070,103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[92.3454, 51.8951, 46.656, 42.5085, 27.8181, 2...","[232.71, 152.88, 132.2, 118.21, 53.07, 69.46, ...","[17.73, 17.64, 17.46, 17.35, 17.14, 16.99, 16....","[18.23, 18.14, 18.02, 17.93, 18.33, 19.89, 20....","[0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.0...","[104.76, 96.46, 128.23, 128.2, 113.63, 85.32, ...",24026,117,"[178.41, 241.21, 151.74, 0.0, 184.48, 242.99, ..."
...,...,...,...,...,...,...,...,...,...
37044,"[353.3932, 382.6572, 430.3689, 433.7981, 460.0...","[755.06, 742.34, 790.33, 773.86, 791.98, 703.8...","[324.56, 324.56, 324.56, 324.56, 324.56, 324.5...","[16.12, 16.13, 16.3, 16.31, 16.3, 16.3, 16.11,...","[0.02, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.42...",26081,67,"[0.0, 0.0, 0.0, 143.76, 111.39, 209.05, 148.18..."
37045,"[34.3281, 10.3602, 9.1293, 7.6454, 10.2183, 13...","[107.14, 31.96, 35.2, 37.22, 52.73, 50.69, 41....","[39.02, 39.67, 40.05, 39.09, 38.69, 39.59, 39....","[39.79, 40.3, 41.93, 41.37, 40.44, 41.48, 42.4...","[0.07, 0.05, 0.04, 0.02, 0.01, 0.01, 0.01, 0.0...","[448.04, 366.33, 318.85, 503.17, 327.75, 358.7...",10203,110,"[445.67, 487.56, 451.39, 491.9, 568.12, 436.23..."
37046,"[6.2295, 58.4111, 84.6045, 51.4788, 59.7765, 5...","[0.0, 67.83, 217.08, 150.25, 163.84, 167.28, 2...","[31.1, 31.44, 31.22, 31.09, 31.0, 30.84, 30.62...","[37.39, 37.47, 37.48, 37.3, 37.16, 36.92, 36.7...","[51.17, 29.55, 0.01, 0.01, 0.01, 0.01, 0.01, 0...","[0.0, 0.0, 0.0, 122.54, 134.98, 64.59, 53.01, ...",15671,107,"[0.0, 0.0, 0.0, 32.49, 0.0, 0.0, 0.0, 0.0, 0.0..."
37047,"[252.436, 211.7087, 228.0991, 174.6769, 125.0,...","[650.06, 490.62, 500.19, 466.35, 355.16, 377.4...","[20.79, 20.69, 20.59, 20.41, 20.27, 20.23, 20....","[37.39, 37.24, 37.19, 37.09, 37.09, 37.01, 36....","[0.01, 3.14, 3.09, 1.87, 3.06, 3.1, 2.92, 3.78...","[343.52, 324.64, 462.57, 798.65, 644.13, 441.4...",19585,125,"[449.34, 276.97, 409.61, 451.46, 437.52, 451.2..."


In [17]:
df_train['Pab1_seq'] = df_train['Pab1_seq'].apply(func)

In [18]:
df_train['Pab1_seq'] = df_train.apply(
  lambda x: [round(np.sin((2*np.pi*r)/360),4) for r in x['Pab1_seq']],
  axis=1,
)

In [19]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
def my_scale(lst):
    n = len(lst)
    x = np.array(lst).reshape(-1,1)
    scaler_robust = RobustScaler()
    scaler_robust.fit(x)
    k = scaler_robust.transform(x)
    kk = k.reshape(n)
    return list(np.round(kk, 4))

In [20]:
df_train['Patv_seq'] = df_train['Patv_seq'].apply(func)
df_train['Etmp_seq'] = df_train['Etmp_seq'].apply(func)
df_train['Itmp_seq'] = df_train['Itmp_seq'].apply(func)
# df_train['Patv_space'] = df_train['Patv_space'].apply(func)

In [21]:
df_train['Wspd_seq'] = df_train['Wspd_seq'].apply(lambda x : my_scale(x))
df_train['Patv_seq'] = df_train['Patv_seq'].apply(lambda x : my_scale(x))
df_train['Etmp_seq'] = df_train['Etmp_seq'].apply(lambda x : my_scale(x))
df_train['Itmp_seq'] = df_train['Itmp_seq'].apply(lambda x : my_scale(x))
# df_train['Patv_space'] = df_train['Patv_space'].apply(lambda x : my_scale(x))

In [22]:
df_train

,Wspd_seq,Patv_seq,Etmp_seq,Itmp_seq,Pab1_seq,target,index,TurbID,Patv_space
0,"[-0.1743, -0.1737, -0.1715, -0.1662, -0.1516, ...","[-0.1833, -0.1833, -0.1833, -0.1833, -0.1833, ...","[-0.4851, -0.4942, -0.489, -0.4773, -0.4773, -...","[-0.3207, -0.3332, -0.3567, -0.3693, -0.4194, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1422.04, 1519.51, 1501.53, 1519.98, 1519.36, ...",21207,14,"[1194.55, 1411.22, 1519.46, 1515.86, 1447.61, ..."
1,"[-0.3364, -0.3964, -0.293, -0.1186, -0.318, -0...","[-0.3675, -0.3675, -0.3675, -0.3675, -0.3675, ...","[0.5691, 0.426, 0.4167, 0.4167, 0.4102, 0.4037...","[0.5655, 0.8317, 1.0148, 1.003, 0.9741, 0.9487...","[0.6859, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5885,...","[209.79, 247.88, 371.06, 183.73, 194.71, 132.3...",9625,118,"[155.11, 190.08, 153.1, 197.71, 143.08, 155.7,..."
2,"[0.1565, 0.2133, 0.1429, 0.1497, 0.0, -0.0594,...","[0.5605, 0.5629, 0.5083, 0.4499, 0.3879, 0.391...","[-1.5684, -1.5704, -1.5824, -1.5884, -1.6224, ...","[-1.7724, -1.777, -1.809, -1.8205, -1.8411, -1...","[-0.0347, -0.0347, -0.0347, -0.0347, -0.0347, ...","[80.74, 77.03, 100.68, 1.18, 0.0, 0.0, 0.0, 0....",20781,126,"[100.73, 86.67, 83.81, 92.14, 107.54, 117.99, ..."
3,"[2.4374, 2.041, 2.022, 1.4162, 1.7261, 1.6078,...","[0.9856, 0.9944, 1.3484, 1.2951, 1.4136, 1.330...","[0.6354, 0.6546, 0.6565, 0.6528, 0.6619, 0.683...","[0.5507, 0.5682, 0.5992, 0.6176, 0.6408, 0.660...","[0.1625, 0.1463, 0.0839, 0.0333, 0.0379, 0.035...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",16070,103,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[1.9007, 0.8426, 0.7056, 0.5971, 0.2128, 0.071...","[2.2655, 1.4883, 1.287, 1.1508, 0.5166, 0.6762...","[0.2288, 0.2155, 0.1887, 0.1724, 0.1412, 0.118...","[-0.1278, -0.1495, -0.1784, -0.2001, -0.1037, ...","[0.0009, 0.0009, 0.0009, 0.0009, 0.0009, 0.000...","[104.76, 96.46, 128.23, 128.2, 113.63, 85.32, ...",24026,117,"[178.41, 241.21, 151.74, 0.0, 184.48, 242.99, ..."
...,...,...,...,...,...,...,...,...,...
37044,"[-0.1038, -0.0443, 0.0528, 0.0598, 0.1133, 0.1...","[0.2628, 0.2462, 0.309, 0.2875, 0.3112, 0.1958...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.66, -0.6574, -0.6121, -0.6094, -0.6121, -0...","[0.0003, 0.0003, 0.0002, 0.0002, 0.0002, 0.000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.42...",26081,67,"[0.0, 0.0, 0.0, 143.76, 111.39, 209.05, 148.18..."
37045,"[0.4636, 0.0118, -0.0114, -0.0393, 0.0092, 0.0...","[0.9904, 0.2954, 0.3254, 0.344, 0.4874, 0.4686...","[0.5474, 0.7004, 0.7899, 0.5639, 0.4697, 0.681...","[0.3869, 0.5679, 1.1464, 0.9476, 0.6176, 0.986...","[0.0012, 0.0009, 0.0007, 0.0003, 0.0002, 0.000...","[448.04, 366.33, 318.85, 503.17, 327.75, 358.7...",10203,110,"[445.67, 487.56, 451.39, 491.9, 568.12, 436.23..."
37046,"[-0.0763, 1.635, 2.4941, 1.4077, 1.6798, 1.605...","[0.0, 0.581, 1.8593, 1.2869, 1.4033, 1.4327, 1...","[0.4809, 0.5269, 0.4971, 0.4795, 0.4674, 0.445...","[0.4828, 0.4946, 0.4961, 0.4695, 0.4489, 0.413...","[0.779, 0.4932, 0.0002, 0.0002, 0.0002, 0.0002...","[0.0, 0.0, 0.0, 122.54, 134.98, 64.59, 53.01, ...",15671,107,"[0.0, 0.0, 0.0, 32.49, 0.0, 0.0, 0.0, 0.0, 0.0..."
37047,"[1.243, 0.9332, 1.0579, 0.6515, 0.2737, 0.3438...","[1.2121, 0.6981, 0.729, 0.6199, 0.2614, 0.3334...","[-0.3457, -0.3646, -0.3834, -0.4173, -0.4437, ...","[0.7514, 0.6676, 0.6397, 0.5838, 0.5838, 0.539...","[0.0002, 0.0548, 0.0539, 0.0326, 0.0534, 0.054...","[343.52, 324.64, 462.57, 798.65, 644.13, 441.4...",19585,125,"[449.34, 276.97, 409.61, 451.46, 437.52, 451.2..."


In [23]:
train_df = df_train[:-3700]

In [24]:
val_df = df_train[-3700:]

In [25]:
cols = [x for x in train_df.columns if 'seq' in x or x=='target' or x=='Patv_space']
for col in cols:
    train_df[col] = train_df[col].apply(lambda x: json.loads(str(x)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
cols = [x for x in val_df.columns if 'seq' in x or x=='target' or x=='Patv_space']
for col in cols:
    val_df[col] = val_df[col].apply(lambda x: json.loads(str(x)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
train_dataset = myDataset(train_df)

train_loader = DataLoader(train_dataset,
                      batch_size=32,
                      shuffle=False,
                      num_workers=0)

In [28]:
val_dataset = myDataset(val_df)

val_loader = DataLoader(val_dataset,
                      batch_size=32,
                      shuffle=False,
                      num_workers=0)

In [29]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [30]:
def train(model, optimizer, train_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    metric = nn.L1Loss().to(device)

    ls = []
    pred_list = []
    label_list = []
    val_loss_lst = []
    val_mae_lst = []
    loss_list = []
    early_stopping = EarlyStopping(patience = 5, verbose = True)
    for epoch in range(100):
        model.train()

        for seq, space_data, label in tqdm(train_loader):
            seq = seq.type(torch.float32).to(device)

            space_data = torch.reshape(space_data, (-1,1,11,11 )).to(device)
            space_data = space_data.type(torch.float32)
    
            label = label.type(torch.float32).to(device)

            pred = model(seq, space_data)

            loss = criterion(pred.squeeze()/1000, label/1000)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            pred_list.extend(pred.squeeze().cpu().detach().numpy())
            label_list.extend(label.squeeze().cpu().detach().numpy())
            
            del pred 
            del seq
            del space_data
            del label
            
        for val_seq, val_space_data, val_label in tqdm(val_loader):
            val_seq = val_seq.type(torch.float32).to(device)
            val_space_data = torch.reshape(val_space_data, (-1,1,11,11 )).to(device)
            val_space_data = val_space_data.type(torch.float32)
    
            val_label = val_label.type(torch.float32)
            val_pred = model(val_seq, val_space_data).cpu()


            val_loss = criterion(val_pred.squeeze()/1000, val_label/1000)
            with torch.no_grad():
                val_mae = metric(val_pred.squeeze(), val_label)
            val_loss_lst.append(val_loss.item())
            val_mae_lst.append(val_mae)

            
        val_mae = np.average(val_mae_lst)
        
        early_stopping(val_mae,model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break


        total_loss = torch.mean(torch.tensor(loss_list, device = 'cuda'))
#         model.eval()

        print(
            f'Epoch: {epoch} Loss: {total_loss}'
        )
        torch.cuda.empty_cache()
        
    torch.save(model.state_dict(), './ourbest2batch32shuffleF3sin_data_normal_lr05scaling22yeswspdscale.pdparams', _use_new_zipfile_serialization=False)
    print('Model Saved.')
    
    return model

In [31]:
model = GRU()
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005)
train(model, optimizer, train_loader, device)

  0%|                                                                                          | 0/1043 [00:00<?, ?it/s]

cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])


  0%|▏                                                                               | 3/1043 [00:13<1:02:09,  3.59s/it]

cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])


  1%|▊                                                                                | 11/1043 [00:14<10:27,  1.64it/s]

...

 19%|██████████████▌                                                                | 193/1043 [00:15<00:07, 106.30it/s]

cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch

 21%|█████████████████                                                               | 222/1043 [00:16<00:59, 13.72it/s]

cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch.Size([32, 288, 2])
cnn_out
torch.Size([32, 64, 3, 3])
torch

KeyboardInterrupt: 